In [ ]:
####################### AUTO LOGIN #####################
import os
os.system('clear')
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning) 
print('Starting Strategy...')
import json
import login
from login import *
import pandas as pd
import joblib
import threading
import ta
import requests
import numpy as np
pd.set_option('plotting.backend', 'pandas_bokeh')
import pandas_bokeh
pandas_bokeh.output_notebook()
import h2o
hh=h2o.init(ip='127.0.0.1',port='12349')
n50 = pd.read_csv('n50.csv')
nfifty_tokens_list = n50['instrument_token'].to_numpy().tolist()
n100 = pd.read_csv('n100.csv')
n100_tokens_list = n100['instrument_token'].to_numpy().tolist()

In [ ]:
order_type = 'MIS' 
mymargin = 45000
interval='5minute'
daysdiff = 10
bkg = 0.0
isneg = 1

todaydt=datetime.date.today()
hud_ago=todaydt-datetime.timedelta(days=daysdiff)
to_date=datetime.date.isoformat(todaydt)
from_date=datetime.date.isoformat(hud_ago)

print('from_date : '+str(from_date))
print('to_date : '+str(to_date))

In [ ]:
# one stock predict and place order

saved_model = h2o.load_model('../XGBoost_2_AutoML_20210109_204532')
saved_model2 = h2o.load_model('../GBM_grid__1_AutoML_20210109_115241_model_1')
saved_model3 = h2o.load_model('../DRF_1_AutoML_20210111_165257')
saved_model4 = h2o.load_model('../GBM_3_AutoML_20210113_220248')

def one_stk_fun(insttkn,tradeon=False):
    '''one_stk_fun(insttkn,tradeon=False)'''
    old_lst = kite.historical_data(insttkn, from_date, to_date, interval,continuous=False)

    mydf = pd.DataFrame(old_lst)
    mydf = mydf.fillna(0)
    mydf = ta.add_all_ta_features(mydf, open="open", high="high", low="low", close="close", volume="volume")
    mydf = mydf.fillna(0)[150:]

    hf = h2o.H2OFrame(mydf)

    prd1 = saved_model.predict(hf)
    ct1 = prd1.concat([hf])
    prd2 = saved_model2.predict(ct1)
    ct2 = prd2.concat([ct1])
    prd3 = saved_model3.predict(ct2)

    cls = ct2.col_names

    for n, i in enumerate(cls):
        if (i == 'predict0'):
            cls[n] = 'predict0.1'
        if (i == 'predict'):
            cls[n] = 'predict0'
        if (i == 'p-1'):
            cls[n] = 'p-10'
        if (i == 'p0'):
            cls[n] = 'p00'
        if (i == 'p1'):
            cls[n] = 'p10'

    ct2.set_names(cls)

    ct3 = prd3.concat([ct2])
    prd4 = saved_model4.predict(ct3)
    ct4 = prd4.concat([ct3])

    os.system('clear')
    data_as_df = h2o.as_list(ct4)  

    sel_data = data_as_df[['predict','predict0','predict0.1','date','close']] #,'next_close'

    last_isbullbear = 0

    total_gained_pt = []
    total_gained_pt_total = 0
    total_gained_pt_lst = []

    for ind in sel_data.index:

        is_bullbear = 0

        if(ind<(len(sel_data.index)-1)):
            if(sel_data['close'][ind]<sel_data['close'][ind+1]):
                is_bullbear=1
            elif(sel_data['close'][ind]>sel_data['close'][ind+1]):
                is_bullbear=-1

        pred_bullbear = 0

        if(ind<(len(sel_data.index)-1)):
            if(sel_data['predict'][ind]<sel_data['close'][ind]):
                pred_bullbear=-1
            elif(sel_data['predict'][ind]>sel_data['close'][ind]):
                pred_bullbear=1

        pred_true = -1

        if(pred_bullbear==is_bullbear):
            pred_true = 1

        gained_pt=0
        if(ind<(len(sel_data.index)-1)):
            gained_pt = pred_true*abs(sel_data['close'][ind]-sel_data['close'][ind+1])

        is_global_avail = 'isneg'+str(insttkn) in globals() 

        if(is_global_avail):
            gained_pt = globals()['isneg'+str(insttkn)]*gained_pt
        else:
            gained_pt = gained_pt*isneg


    #         gained_pt = gained_pt*isneg
        if(ind>0 and (last_isbullbear!=is_bullbear)):
            gained_pt = gained_pt+bkg

        total_gained_pt.append(gained_pt)
        total_gained_pt_total = total_gained_pt_total + gained_pt
        total_gained_pt_lst.append(total_gained_pt_total)
        last_isbullbear = is_bullbear

    this_mean_val = np.array(total_gained_pt_lst).mean()
    gdptdf = pd.DataFrame({'total_gained_pt':total_gained_pt,'total_gained_pt_lst':total_gained_pt_lst})
#     gdptdf.plot()

    nw = datetime.datetime.now()
    last_candle_time =  nw.replace(hour=15, minute=30, second=0, microsecond=0)
    zero_hour =  nw.replace(hour=0, minute=0, second=0, microsecond=0)
    
    selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'{0}'.format(str(nw.hour).zfill(2))+':' +'{0}'.format(str(nw.minute-5).zfill(2))+':'+'00+05:30'
    
    if(nw<last_candle_time):
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
        
    if(nw>zero_hour):
        selected_time_str = str(nw.year) + '-' + '{0}'.format(str(nw.month).zfill(2))+ '-' +'{0}'.format(str(nw.day-1).zfill(2))+ ' ' +'15'+':' +'25'+':'+'00+05:30'
    
    this_qt = sel_data.loc[sel_data['date'] == selected_time_str].to_numpy()[0]
    
    pred_close = this_qt[0]
    this_close = this_qt[4]
    this_time = this_qt[3]

    this_mean_percent = (this_mean_val/this_close)*100

    if(this_mean_percent<0):
        globals()['isneg'+str(insttkn)]=-1
    else:
        globals()['isneg'+str(insttkn)]=1

    current_bullbear = 0

    if(pred_close>this_close):
        current_bullbear=1

    elif(pred_close<this_close):
        current_bullbear=-1

    print('=============================')
    print('current_bullbear : '+str(current_bullbear))
    print('pred_close : '+str(pred_close))
    print('this_close : '+str(this_close))
    print('this_mean_percent : '+str(this_mean_percent))
    print('this_time : '+str(this_time))
    print('=============================')

    if(tradeon):
        is_global_avail = 'isneg'+str(insttkn) in globals() 
        this_neg_local = 0
        if(is_global_avail):
            this_neg_local = globals()['isneg'+str(insttkn)]
        else:
            this_neg_local = isneg
        print('isneg : '+str(this_neg_local))
        this_qt = new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['insttkn'] == insttkn, 'full_qty'].to_numpy()[0] 
        this_symbol = new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['insttkn'] == insttkn, 'tradingsymbol'].to_numpy()[0] 
        final_qt = int(this_neg_local*current_bullbear*this_qt)
        print('Placing order for : '+str(insttkn)+' : '+ this_symbol+' : qty : '+str(final_qt))
        ckqnt(0,final_qt,this_symbol)

    globals()['df'+str(insttkn)] = pd.DataFrame({insttkn:total_gained_pt_lst})
    globals()['df'+str(insttkn)] = (globals()['df'+str(insttkn)]/this_close)*100

    return globals()['df'+str(insttkn)]

In [ ]:
# Select Target Token and Symbol Name
def decide_target(nfifty_tokens_list):
    '''decide_target(nfifty_tokens_list)'''
    global mynp,last_mean,combined_df,combined_df_mean_sorted,symbol_ip
    print('Deciding From Date : '+from_date)
    print('Deciding To Date : '+to_date)
    
    combined_df = pd.DataFrame()
    counter1 = 1
    for iii in nfifty_tokens_list:
        
        symbol_name1 = n50.loc[n50['instrument_token'] == iii, 'tradingsymbol'].to_numpy()[0]
        print()
        print('============================================================================')
        print('Analyzing Token : '+str(iii)+ ' : '+str(symbol_name1)+ ' : '+str(counter1)+'/'+str(len(nfifty_tokens_list)))
        print('============================================================================')
        counter1 = counter1+1
        one_stk_fun(iii,tradeon=False)
        combined_df = pd.concat([combined_df,globals()['df'+str(iii)]], axis=1)

    combined_df.plot()
    os.system('clear')
    
    mynp = combined_df.iloc[[-1]].to_numpy()[0]
    mynp = np.nan_to_num(mynp)
    print('Nifty50 Mean : '+str(np.mean(mynp)))
    print('Nifty50 Sum : '+str(np.sum(mynp)))

    insttkn = 0
    last_mean = 0
    
    selected_tkn_list = []
    selected_symbol_list = []
    selected_mean_list = []

    for ij in range(len(nfifty_tokens_list)):
        
        this_mean = combined_df[nfifty_tokens_list[ij]].to_numpy().mean()

        if(this_mean>last_mean):
            insttkn= nfifty_tokens_list[ij]
            last_mean = this_mean

    combined_df_mean = pd.DataFrame({'insttkn':combined_df.mean().keys(),
                                     'combined_df_mean':combined_df.mean().to_numpy(),
                                    })
    combined_df_mean_sorted = combined_df_mean.sort_values('combined_df_mean',ascending=False)[0:5]
#     print(combined_df_mean_sorted)
    symbol_ip = n50.loc[n50['instrument_token'] == insttkn, 'tradingsymbol'].to_numpy()[0]

    print('Success Rate Mean : '+str(last_mean))
    print('Selected Token : '+str(insttkn))
    print('Selected Symbol : '+str(symbol_ip))
    return {'insttkn':insttkn,'symbol_ip':symbol_ip,'mean':last_mean,
            'combined_df_mean_sorted':combined_df_mean_sorted}

In [ ]:
# base functions : getquant, placeneworder, ckqnt

myquantity = 0
def getquant():
    global myquantity,order_type
    allpos = kite.positions()['net']
    for iii in new_combined_df_mean_sorted['tradingsymbol']:
        for i in range(len(allpos)):
            if(allpos[i]['tradingsymbol']==iii and  allpos[i]['product']== order_type):
                new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['tradingsymbol'] == iii, 'my_qty']=allpos[i]['quantity']
    #             myquantity = allpos[i]['quantity']
    #             print('My Quantity : ' + str(allpos[i]['quantity']))


stop_trading = False
ex_type = kite.EXCHANGE_NSE

def placeneworder(quantdiff,price_ip,symb_name):
    global stop_trading,order_type,ex_type
    if(quantdiff>0 and stop_trading==False):
        try:
#             stop_trading = True
            order_id= kite.place_order(tradingsymbol=symb_name,
                        exchange=ex_type,
                        transaction_type=kite.TRANSACTION_TYPE_BUY,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
#             stop_trading = False
        except Exception as e:
            print(e)
            getquant()
    if(quantdiff<0 and stop_trading==False):
        try:
#             stop_trading = True
            order_id= kite.place_order(tradingsymbol=symb_name,
                        exchange=ex_type,
                        transaction_type=kite.TRANSACTION_TYPE_SELL,
                        quantity=abs(quantdiff),
#                             price=price_ip,
                        order_type=kite.ORDER_TYPE_MARKET,
                        variety = kite.VARIETY_REGULAR,
                        product=kite.PRODUCT_MIS if order_type=='MIS' else kite.PRODUCT_NRML)
            getquant()
#             stop_trading = False
        except Exception as e:
            print(e)
            getquant()

def ckqnt (orderprice,orderquant,symb_name):
    global myquantity
    
    this_my_qty = new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['tradingsymbol'] == symb_name, 'my_qty'].to_numpy()[0]
#     this_order_qty = new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['tradingsymbol'] == symb_name, 'full_qty'].to_numpy()[0]
    
    quantdiff = int(orderquant - this_my_qty)
    print('-------------------')
    print('symb_name : '+str(symb_name))
    print('orderquant : '+str(orderquant))
    print('this_my_qty : '+str(this_my_qty))
    print('placeorder quantdiff : '+str(quantdiff))
    print('-------------------')
    placeneworder(quantdiff,orderprice,symb_name)
    return myquantity

def print_params():
    print('=============================')
    print('Nifty50 Mean : '+str(np.mean(mynp)))
    print('Nifty50 Sum : '+str(np.sum(mynp)))
    print('Success Rate Mean : '+str(last_mean))
    print('Selected Token : '+str(insttkn))
    print('Selected Symbol : '+str(symbol_ip))
    print('Order Type : '+str(order_type))
    print('My Margin : '+str(mymargin))
    print('Interval : '+str(interval))
    print('MIS Qty : '+str(full_qty))
    print('=============================')

In [ ]:
############### Calculate MIS Qty ############

url = 'https://api.kite.trade/margins/equity'
margin_data = requests.get(url, timeout=2.50)
margin_map = json.loads(margin_data.content.decode('utf-8'))
margin_map_df = pd.DataFrame(margin_map)

ipmap = decide_target(nfifty_tokens_list)
ipmap = decide_target(nfifty_tokens_list)

symbol_list = []

for one_inst in combined_df_mean_sorted['insttkn'].to_numpy():
    this_symbolnameip = n50.loc[n50['instrument_token'] == one_inst, 'tradingsymbol'].to_numpy()[0]
    symbol_list.append(this_symbolnameip)
    
symboldf = pd.DataFrame({'tradingsymbol':symbol_list})

combined_df_mean_sorted = combined_df_mean_sorted.reset_index(drop=True)    
symboldf.reset_index() 

new_combined_df_mean_sorted = pd.concat([combined_df_mean_sorted, symboldf], axis=1)
# print(new_combined_df_mean_sorted)

margin_per_stock = mymargin/5

new_combined_df_mean_sorted['full_qty']=0
new_combined_df_mean_sorted['my_qty']=0

for one_symbol in new_combined_df_mean_sorted['tradingsymbol']:

    this_mis_multiplier = margin_map_df.loc[margin_map_df['tradingsymbol'] == one_symbol, 'mis_multiplier'].to_numpy()[0]
    this_token = new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['tradingsymbol'] == one_symbol,'insttkn'].to_numpy()[0]
    this_quote = kite.quote([this_token])
    this_ltp = this_quote[str(this_token)]['last_price']
    this_mis_margin = this_ltp/this_mis_multiplier
    
    this_full_qty = int(margin_per_stock/this_mis_margin)
    
    new_combined_df_mean_sorted.loc[new_combined_df_mean_sorted['tradingsymbol'] == one_symbol, 'full_qty'] = this_full_qty

print(new_combined_df_mean_sorted)

In [ ]:
new_combined_df_mean_sorted['tradingsymbol']

In [ ]:
# 5 minute time loop with predict/order Fn
import time
last_min = 0
getquant()
now = datetime.datetime.now()
start_time = now.replace(hour=10, minute=0, second=0, microsecond=0)
stop_time = now.replace(hour=14, minute=45, second=0, microsecond=0)

while(1):
    minute=datetime.datetime.now().minute
    
    now = datetime.datetime.now()
    if((now < start_time) and (now > stop_time)):
        if(minute%5==0 and(minute!=last_min)):
            os.system('clear')
            for one_inst_ in new_combined_df_mean_sorted['insttkn']:
                x = threading.Thread(target=one_stk_fun, args=([one_inst_,True]))
                x.start()
    #             one_stk_fun(one_inst_,True)
            last_min=minute
            print(new_combined_df_mean_sorted)
        time.sleep(0.1)
    else:
        for onestknm in new_combined_df_mean_sorted['tradingsymbol']:
            ckqnt (0,0,onestknm)
        break
        

In [ ]:
# one_stk_fun(1270529,tradeon=False)
# # 969473
# # 81153

In [ ]:
# import py_compile
# py_compile.compile('./mystrat.py')

In [ ]:
# ipynb-py-convert stock_predict_pipe_11_1_2021.ipynb mystrat.py
# python3 -m compileall mystrat.py

In [ ]:
todaydt=datetime.date.today()

for i in range(5):
    
#     todaydt=datetime.date.today()-datetime.timedelta(days=((i*5)-2))
#     hud_ago=todaydt-datetime.timedelta(days=2)
        
    to_proloss_date=datetime.date.isoformat(todaydt-datetime.timedelta(days=(i)))
    from_proloss_date=datetime.date.isoformat(todaydt-datetime.timedelta(days=(i+4)))
    
#     print('to_proloss_date '+str(to_proloss_date))
#     print('from_proloss_date '+str(from_proloss_date))
    
######################################    

#     todaydt=datetime.date.today()-datetime.timedelta(days=(i+1)*5)
#     hud_ago=todaydt-datetime.timedelta(days=4)
    
    to_calc_taget_date=datetime.date.isoformat(todaydt-datetime.timedelta(days=(i+5)))
    from_calc_taget_date=datetime.date.isoformat(todaydt-datetime.timedelta(days=(i+9)))
    
#     print('to_calc_taget_date '+str(to_calc_taget_date))
#     print('from_calc_taget_date '+str(from_calc_taget_date))
#     print('------------------------------------')
################################    

    to_date = to_calc_taget_date
    from_date = from_calc_taget_date
    map1 = decide_target(nfifty_tokens_list)
    
# #################################

    to_date = to_proloss_date
    from_date = from_proloss_date
    
    tknip = map1['insttkn']
    print('*********************************************')
    print('++++++++++++ Result '+str(i+1)+'+++++++++++++')
    print('from_calc_taget_date '+str(from_calc_taget_date))
    print('to_calc_taget_date '+str(to_calc_taget_date))
    print('from_proloss_date '+str(from_proloss_date))
    print('to_proloss_date '+str(to_proloss_date))
    print('---------------------------------------------')
    one_stk_fun(tknip,False)
    print('*********************************************')
    
    print()    

In [ ]:
# ############### Calculate MIS Qty for one stock ############

# url = 'https://api.kite.trade/margins/equity'
# margin_data = requests.get(url, timeout=2.50)
# margin_map = json.loads(margin_data.content.decode('utf-8'))
# margin_map_df = pd.DataFrame(margin_map)
# margin_map_df

# ipmap = decide_target(nfifty_tokens_list)
# ipmap = decide_target(nfifty_tokens_list)

# symbol_ip = ipmap['symbol_ip']
# insttkn = ipmap['insttkn']

# mis_margin = margin_map_df.loc[margin_map_df['tradingsymbol'] == symbol_ip, 'mis_multiplier'].to_numpy()[0]

# myqt = kite.quote([insttkn])
# ltp = myqt[str(insttkn)]['last_price']
# mis_margin_ip = ltp/mis_margin

# full_qty = int(mymargin/mis_margin_ip)
# full_qty

In [ ]:
# Datetime features

# df[‘ScheduledDay_year’] = df[‘ScheduledDay’].dt.year
# df[‘ScheduledDay_month’] = df[‘ScheduledDay’].dt.month
# df[‘ScheduledDay_week’] = df[‘ScheduledDay’].dt.week
# df[‘ScheduledDay_day’] = df[‘ScheduledDay’].dt.day
# df[‘ScheduledDay_hour’] = df[‘ScheduledDay’].dt.hour
# df[‘ScheduledDay_minute’] = df[‘ScheduledDay’].dt.minute
# df[‘ScheduledDay_dayofweek’] = df[‘ScheduledDay’].dt.dayofweek

# pandas.Series.asfreq
# pandas.Series.asof
# pandas.Series.shift
# pandas.Series.first_valid_index
# pandas.Series.last_valid_index
# pandas.Series.resample
# pandas.Series.tz_convert
# pandas.Series.tz_localize
# pandas.Series.at_time
# pandas.Series.between_time
# pandas.Series.tshift
# pandas.Series.slice_shift

# pandas.Series.dt.date
# pandas.Series.dt.time
# pandas.Series.dt.year
# pandas.Series.dt.month
# pandas.Series.dt.day
# pandas.Series.dt.hour
# pandas.Series.dt.minute
# pandas.Series.dt.second
# pandas.Series.dt.microsecond
# pandas.Series.dt.nanosecond
# pandas.Series.dt.week
# pandas.Series.dt.weekofyear
# pandas.Series.dt.dayofweek
# pandas.Series.dt.weekday
# pandas.Series.dt.dayofyear
# pandas.Series.dt.quarter
# pandas.Series.dt.is_month_start
# pandas.Series.dt.is_month_end
# pandas.Series.dt.is_quarter_start
# pandas.Series.dt.is_quarter_end
# pandas.Series.dt.is_year_start
# pandas.Series.dt.is_year_end
# pandas.Series.dt.is_leap_year
# pandas.Series.dt.daysinmonth
# pandas.Series.dt.days_in_month
# pandas.Series.dt.tz
# pandas.Series.dt.freq
# pandas.Series.dt.to_period
# pandas.Series.dt.to_pydatetime
# pandas.Series.dt.tz_localize
# pandas.Series.dt.tz_convert
# pandas.Series.dt.normalize
# pandas.Series.dt.strftime
# pandas.Series.dt.round
# pandas.Series.dt.floor
# pandas.Series.dt.ceil
# pandas.Series.dt.month_name
# pandas.Series.dt.day_name
# pandas.Series.dt.days
# pandas.Series.dt.seconds
# pandas.Series.dt.microseconds
# pandas.Series.dt.nanoseconds
# pandas.Series.dt.components
# pandas.Series.dt.to_pytimedelta
# pandas.Series.dt.total_seconds

In [ ]:
# from notebook.auth import passwd; passwd(passphrase='haj123', algorithm='sha1')